In [7]:
from langchain_neo4j import Neo4jGraph
from langchain_core.prompts import ChatPromptTemplate

In [8]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile", api_key='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')

In [9]:
graph = Neo4jGraph(
    url="neo4j+s://00601af8.databases.neo4j.io", username="neo4j", password="oIBWHe2qEKESesEC1pBzwXL041YajSkKN5BIacTIIE0"
)

In [10]:
graph

In [11]:
def extract_player_social_diversity_features(player_id: str) -> dict:
    """
    Extracts features from the knowledge graph for a given player.
    """
    query = f"""
    MATCH (p:Player {{Actor: toInteger('{player_id}')}})
    RETURN
        p.Actor AS player_id,
        p.A_Acc AS a_acc,
        p.Social_diversity AS social_diversity
    """
    results = graph.query(query)
    if results:
        return results[0]
    else:
        return {}

In [22]:
prompt = ChatPromptTemplate.from_template(
"""
<|system|>
You are an expert game analyst tasked with identifying bots in an online game by analyzing player statistics. Your role is to generate an anomaly score (0-100, higher score means more likely to be a bot) and provide concise reasoning based on the player data.

Here is how to interpret each player statistic. Provide context for how each statistic informs your score. Focus on how these characteristics point to a bot.

- **Actor**: The player's unique identifier.
- **A_Acc**: Account ID of the player; may provide clues about account creation patterns.
- **Social_Diversity**: Represents the extent of a player's interactions within the game, including guild participation, friendships, trades, chats, and group activities, ranges between 0 - 2. A higher value indicates a more varied and diverse social presence.

Consider:

1. A very low Social Diversity Score increases the likelihood of bot-like behavior, as it suggests minimal engagement with different players and activities.
2. Players with consistently limited interactions or repetitive engagement patterns are more likely to be automated rather than naturally social participants.
3. Higher Social Diversity Scores generally indicate a more dynamic and organic playstyle, as players engage with different groups, trade partners, and social features over time.
4. While some legitimate players may have low social diversity due to playstyle preferences, inactivity, or being new to the game, consistently low diversity over extended periods is more commonly seen in bots.


Based on those descriptions, you will be given the following information and need to determine these two things

1.  Determine the **anomaly score** (0-100): A higher score indicates a greater likelihood of the player being a bot. This MUST be provided

2.  Provide a **detailed explanation** of your reasoning. This MUST be provided

Your final output **MUST** follow this structure:

Anomaly Score: [your anomaly score 0-100]
Reasoning: [Your reasoning based on patterns from the statistics. Be as clear as possible]

Respond concisely and directly.

<|user|>
Here is the player data:

Actor: {actor}
A_Acc: {a_acc}
Social_diversity: {social_diversity}

Anomaly Score:
Reasoning:
""")

In [17]:
def assess_bot_likelihood(player_data) -> tuple[int, str, str]:
    """
    Assesses the likelihood of a player being a bot using LLM and extracts score and reasoning.
    Returns a tuple of (anomaly_score, reasoning, full_analysis).
    """
    # Format the prompt
    formatted_prompt = prompt.format_messages(
        actor=player_data['player_id'],
        a_acc=player_data['a_acc'],
        social_diversity=player_data['social_diversity']
    )

    # Call the LLM directly
    response = llm.invoke(formatted_prompt)
    full_analysis = response.content

    # Extract Anomaly Score and Reasoning
    try:
        score_line = next(line for line in full_analysis.split('\n') if "Anomaly Score" in line)
        anomaly_score = int(score_line.split(":")[1].strip())

        reasoning = "\n".join(full_analysis.split('\n')[1:])  # All lines after the score
    except Exception as e:
        anomaly_score = None
        reasoning = f"Could not reliably parse LLM response: {str(e)}"

    return anomaly_score, reasoning, full_analysis

In [18]:
def generate_bot_report(player_ids: list[str]) -> list[dict]:
    """
    Generates a report for a list of player IDs, including anomaly scores and reasoning.
    """
    report = []
    for player_id in player_ids:
        player_data = extract_player_social_diversity_features(player_id)
        if player_data:
            anomaly_score, reasoning, full_analysis = assess_bot_likelihood(player_data)

            report.append({
                "player_id": player_id,
                "anomaly_score": anomaly_score,
                "reasoning": reasoning,
                #"full_analysis": /full_analysis  # Optional - adds full LLM output
            })
        else:
            report.append({
                "player_id": player_id,
                "anomaly_score": None,
                "reasoning": "No data found for this player.",
                "full_analysis": None
            })
    return report

In [ ]:
player_ids_to_check = ['6187','7505','315285','316386']  # Replace with actual player IDs
bot_report = generate_bot_report(player_ids_to_check)

for record in bot_report:
    print(f"Player ID: {record['player_id']}")
    print(f"Anomaly Score: {record['anomaly_score']}")
    # print(f"Reasoning: {record['reasoning']}")
    # print("-" * 40)

Player ID: 6187
Anomaly Score: 90
Reasoning: Reasoning: The player's Social Diversity Score of 0.0 indicates a complete lack of interaction with other players, guilds, or social features within the game. This extremely low score, combined with the potential for patterned account creation suggested by the A_Acc, strongly suggests bot-like behavior. Legitimate players typically exhibit some level of social engagement over time, whereas a score of 0.0 implies a static, potentially automated playstyle. The absence of any social diversity is a significant indicator of a bot, as naturally social players tend to engage in various activities and interactions, leading to higher diversity scores.
----------------------------------------
Player ID: 7505
Anomaly Score: 20
Reasoning: Reasoning: The player's Social Diversity Score of 0.788 is relatively high, indicating a varied and dynamic social presence within the game. This suggests engagement with different players, groups, and activities, whic